In [1]:
import pandas as pd
import numpy as np
from pyranges import PyRanges


<h1> T2T CHM13

In [114]:
## Anotations
t2t_genes = pd.read_csv('../annotation_files/T2T-chm13-catLiftOffGenes.csv')
t2t_telomeres = pd.read_csv('../annotation_files/chm13v2.0_telomere.bed', sep='\t', names=['Chromosome','Start', 'End'])
t2t_sds = pd.read_csv('../annotation_files/chm13v2.0_SD.full.bed', sep='\t')
t2t_sds = t2t_sds[['#chr1', 'start1', 'end1']]
## Read and merge files 
anotation = pd.read_csv('../data/SGDP_anotation.csv', sep=',', encoding='latin-1')

ids_hg19 = pd.DataFrame()
ids_hg19['Sample'] = anotation['3-Illumina_ID']
ids_hg19['Region'] = anotation['10-Region']
t2t_sds.columns =['Chromosome','Start','End']
t2t_sds

/var/folders/05/4pfxtbrs5459ytz3j1jhcgt00000gn/T/ipykernel_91372/1889853910.py:2: DtypeWarning: Columns (23,36) have mixed types. Specify dtype option on import or set low_memory=False.
  t2t_genes = pd.read_csv('../annotation_files/T2T-chm13-catLiftOffGenes.csv')


,Chromosome,Start,End
0,chr1,1,13731
1,chr1,3,140066
2,chr1,16,13731
3,chr1,16,13731
4,chr1,16,198196
...,...,...,...
83511,chrY,62445192,62453929
83512,chrY,62445192,62454747
83513,chrY,62445192,62456950
83514,chrY,62445192,62457871


In [115]:
# is not working 
#chm13 = pd.read_csv('../call_cnvr_chm13/individual_cnv_cnvr.txt', sep='\t')
chm13 = pd.read_csv('/Users/luciabazan/Downloads/AMR_Mayan_LP6005441-DNA_G07', sep='\t', header= None)
chm13.columns = ['Chromosome', 'Start', 'End','CNV_Value','ext1','ext2','ext3','ext4','ext5']
chm13

,Chromosome,Start,End,CNV_Value,ext1,ext2,ext3,ext4,ext5
0,chr1,0,9083,44,0,+,0,0,"0,255,0"
1,chr1,9083,10803,34,0,+,0,0,"32,178,170"
2,chr1,10803,14487,20,0,+,0,0,"0,255,255"
3,chr1,14487,18731,14,0,+,0,0,"255,0,0"
4,chr1,18731,20013,14,0,+,0,0,"255,0,0"
...,...,...,...,...,...,...,...,...,...
1179222,chr22,51301662,51302662,34,0,+,0,0,"32,178,170"
1179223,chr22,51302662,51303662,34,0,+,0,0,"32,178,170"
1179224,chr22,51303662,51306044,36,0,+,0,0,"32,178,170"
1179225,chr22,51306044,51312584,45,0,+,0,0,"0,255,0"


In [116]:
chm13 = chm13.dropna()
chm13['Length_cnvr'] = chm13['End'] - chm13['Start']


#chm13 = chm13[chm13['Sample_ID'] == 'LP6005441-DNA_G07']
chm13['Sample_ID'] = 'LP6005441-DNA_G07'
chm13 = chm13[~chm13['CNV_Value'].isin([2])]

In [117]:
## Keeping only samples that are on old cnvs 
chm13 = chm13.merge(ids_hg19, right_on=['Sample'], left_on=['Sample_ID'])
chm13.drop(columns=['ext1','ext2','ext3','ext4','ext5'], inplace=True)
chm13

,Chromosome,Start,End,CNV_Value,Length_cnvr,Sample_ID,Sample,Region
0,chr1,0,9083,44,9083,LP6005441-DNA_G07,LP6005441-DNA_G07,America
1,chr1,9083,10803,34,1720,LP6005441-DNA_G07,LP6005441-DNA_G07,America
2,chr1,10803,14487,20,3684,LP6005441-DNA_G07,LP6005441-DNA_G07,America
3,chr1,14487,18731,14,4244,LP6005441-DNA_G07,LP6005441-DNA_G07,America
4,chr1,18731,20013,14,1282,LP6005441-DNA_G07,LP6005441-DNA_G07,America
...,...,...,...,...,...,...,...,...
103808,chr22,51301662,51302662,34,1000,LP6005441-DNA_G07,LP6005441-DNA_G07,America
103809,chr22,51302662,51303662,34,1000,LP6005441-DNA_G07,LP6005441-DNA_G07,America
103810,chr22,51303662,51306044,36,2382,LP6005441-DNA_G07,LP6005441-DNA_G07,America
103811,chr22,51306044,51312584,45,6540,LP6005441-DNA_G07,LP6005441-DNA_G07,America


In [118]:
#REMOVE SAMPLES WITH ABERRATIONS

chm13#.value_counts(by=["Sample_ID"])
#chm13.columns = ['Chromosome', 'Start', 'End', 'Sample_ID', 'CNV_start', 'CNV_end', 'Length', 'Length_cnvr','Sample','Region']

chm13= chm13[chm13['Chromosome'] != 'chrM']
chm13 = chm13[chm13['Chromosome'] != 'chrX']
chm13 = chm13[chm13['Chromosome'] != 'chrY']

In [119]:
chm13['Chromosome'] = chm13['Chromosome'].str.replace('chr', '')

In [120]:
#change to df when using telomeres and centromeres filtration
final_cnv = chm13.pivot_table(index=["Chromosome", "Start", "End", 'Length_cnvr'], 
                    columns='Sample', 
                    values='CNV_Value',
                        fill_value=2).reset_index()

final_cnv

Sample,Chromosome,Start,End,Length_cnvr,LP6005441-DNA_G07
0,1,0,9083,9083,44.0
1,1,9083,10803,1720,34.0
2,1,10803,14487,3684,20.0
3,1,14487,18731,4244,14.0
4,1,18731,20013,1282,14.0
...,...,...,...,...,...
92829,9,150593359,150597012,3653,22.0
92830,9,150597012,150599679,2667,26.0
92831,9,150599679,150601193,1514,45.0
92832,9,150601193,150609226,8033,39.0


In [121]:
t2t_deletions_gb = final_cnv[final_cnv['LP6005441-DNA_G07'] < 3]
t2t_deletions_gb_mean = t2t_deletions_gb['LP6005441-DNA_G07'].mean()
t2t_duplications_gb = final_cnv[final_cnv['LP6005441-DNA_G07'] > 1]
t2t_duplications_gb_mean = t2t_duplications_gb['LP6005441-DNA_G07'].mean()
t2t_duplications_gb = PyRanges(t2t_duplications_gb)
t2t_deletions_gb = PyRanges(t2t_deletions_gb)

/opt/anaconda3/lib/python3.11/site-packages/pyranges/methods/init.py:45: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  return {k: v for k, v in df.groupby(grpby_key)}
/opt/anaconda3/lib/python3.11/site-packages/pyranges/methods/init.py:45: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  return {k: v for k, v in df.groupby(grpby_key)}


In [122]:
final_cnv_ranges = PyRanges(final_cnv)


/opt/anaconda3/lib/python3.11/site-packages/pyranges/methods/init.py:45: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  return {k: v for k, v in df.groupby(grpby_key)}


In [123]:
t2t_genes.columns = ['Chromosome', 'Start', 'End', 'name', 'score', 'strand',
       'thickStart', 'thickEnd', 'reserved', 'blockCount', 'blockSizes',
       'chromStarts', 'name2', 'cdsStartStat', 'cdsEndStat', 'exonFrames',
       'txId', 'type', 'geneName', 'geneType', 'sourceGene',
       'sourceTranscript', 'alignmentId', 'alternativeSourceTranscripts',
       'Paralogy', 'UnfilteredParalogy', 'collapsedGeneIds',
       'collapsedGeneNames', 'frameshift', 'exonAnnotationSupport',
       'intronAnnotationSupport', 'transcriptClass', 'transcriptModes',
       'validStart', 'validStop', 'properOrf', 'extra_paralog']

In [124]:
t2t_genes['Chromosome'] = t2t_genes['Chromosome'].str.replace('chr', '')
t2t_genes = PyRanges(t2t_genes)

t2t_telomeres['Chromosome'] = t2t_telomeres['Chromosome'].str.replace('chr', '')
t2t_telomeres = PyRanges(t2t_telomeres)

t2t_sds['Chromosome'] = t2t_sds['Chromosome'].str.replace('chr', '')
t2t_sds = PyRanges(t2t_sds)

/opt/anaconda3/lib/python3.11/site-packages/pyranges/methods/init.py:45: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  return {k: v for k, v in df.groupby(grpby_key)}
/opt/anaconda3/lib/python3.11/site-packages/pyranges/methods/init.py:45: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  return {k: v for k, v in df.groupby(grpby_key)}
/opt/anaconda3/lib/python3.11/site-packages/pyranges/methods/init.py:45: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default

<h2> Genes

In [125]:
#### t2t genes 
coordinates_genes = final_cnv_ranges.nearest(t2t_genes)
coordinates_genes = coordinates_genes.as_df()
coordinates_genes = coordinates_genes[coordinates_genes['Distance'] <1 ]

## T2T telomeres
coordinates_telomeres = final_cnv_ranges.nearest(t2t_telomeres)
coordinates_telomeres = coordinates_telomeres.as_df()
coordinates_telomeres = coordinates_telomeres[coordinates_telomeres['Distance'] <1 ]

#SDs 
coordinates_sds = final_cnv_ranges.nearest(t2t_sds)
coordinates_sds = coordinates_sds.as_df()
coordinates_sds = coordinates_sds[coordinates_sds['Distance'] <1 ]



In [128]:
coordinates_genes

,Chromosome,Start,End,Length_cnvr,LP6005441-DNA_G07,Start_b,End_b,name,score,strand,...,frameshift,exonAnnotationSupport,intronAnnotationSupport,transcriptClass,transcriptModes,validStart,validStop,properOrf,extra_paralog,Distance
0,1,10803,14487,3684,20.0,11135,12457,AL627309.3-1,0,-,...,NaN,NaN,NaN,paralog,Liftoff,NaN,NaN,NaN,True,0
1,1,14487,18731,4244,14.0,14252,21099,AC114498.1-201,0,+,...,NaN,"1,1",1,ortholog,transMap,True,True,True,False,0
2,1,18731,20013,1282,14.0,14252,21099,AC114498.1-201,0,+,...,NaN,"1,1",1,ortholog,transMap,True,True,True,False,0
3,1,20013,28884,8871,13.0,14252,21099,AC114498.1-201,0,+,...,NaN,"1,1",1,ortholog,transMap,True,True,True,False,0
4,1,51814,54076,2262,16.0,52975,54613,AL731661.1-1,0,-,...,NaN,NaN,NaN,paralog,Liftoff,NaN,NaN,NaN,True,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90914,22,51301662,51302662,1000,34.0,51270445,51302683,RPL23AP82-208,0,+,...,NaN,"1,1,1,1","1,1,1",ortholog,transMap,True,True,True,False,0
90915,22,51302662,51303662,1000,34.0,51270445,51302683,RPL23AP82-208,0,+,...,NaN,"1,1,1,1","1,1,1",ortholog,transMap,True,True,True,False,0
90916,22,51303662,51306044,2382,36.0,51296750,51312835,RPL23AP82-205,0,+,...,NaN,"1,1,1","1,1",ortholog,transMap,True,True,True,False,0
90917,22,51306044,51312584,6540,45.0,51296750,51312835,RPL23AP82-205,0,+,...,NaN,"1,1,1","1,1",ortholog,transMap,True,True,True,False,0


In [129]:
coordinates_genes 

,Chromosome,Start,End,Length_cnvr,LP6005441-DNA_G07,Start_b,End_b,name,score,strand,...,frameshift,exonAnnotationSupport,intronAnnotationSupport,transcriptClass,transcriptModes,validStart,validStop,properOrf,extra_paralog,Distance
0,1,10803,14487,3684,20.0,11135,12457,AL627309.3-1,0,-,...,NaN,NaN,NaN,paralog,Liftoff,NaN,NaN,NaN,True,0
1,1,14487,18731,4244,14.0,14252,21099,AC114498.1-201,0,+,...,NaN,"1,1",1,ortholog,transMap,True,True,True,False,0
2,1,18731,20013,1282,14.0,14252,21099,AC114498.1-201,0,+,...,NaN,"1,1",1,ortholog,transMap,True,True,True,False,0
3,1,20013,28884,8871,13.0,14252,21099,AC114498.1-201,0,+,...,NaN,"1,1",1,ortholog,transMap,True,True,True,False,0
4,1,51814,54076,2262,16.0,52975,54613,AL731661.1-1,0,-,...,NaN,NaN,NaN,paralog,Liftoff,NaN,NaN,NaN,True,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90914,22,51301662,51302662,1000,34.0,51270445,51302683,RPL23AP82-208,0,+,...,NaN,"1,1,1,1","1,1,1",ortholog,transMap,True,True,True,False,0
90915,22,51302662,51303662,1000,34.0,51270445,51302683,RPL23AP82-208,0,+,...,NaN,"1,1,1,1","1,1,1",ortholog,transMap,True,True,True,False,0
90916,22,51303662,51306044,2382,36.0,51296750,51312835,RPL23AP82-205,0,+,...,NaN,"1,1,1","1,1",ortholog,transMap,True,True,True,False,0
90917,22,51306044,51312584,6540,45.0,51296750,51312835,RPL23AP82-205,0,+,...,NaN,"1,1,1","1,1",ortholog,transMap,True,True,True,False,0


In [130]:
## Deletions 

In [131]:
#### Genes 
deletions_t2t_coordinates_genes = t2t_deletions_gb.nearest(t2t_genes)
deletions_t2t_coordinates_genes = deletions_t2t_coordinates_genes.as_df()
deletions_t2t_coordinates_genes = deletions_t2t_coordinates_genes[deletions_t2t_coordinates_genes['Distance'] <1 ]
# Mean CN 
deletions_t2t_mean_individual_genes= deletions_t2t_coordinates_genes['LP6005441-DNA_G07'].mean()



#### Telomeres
deletions_t2t_coordinates_telomeres = t2t_deletions_gb.nearest(t2t_telomeres)
deletions_t2t_coordinates_telomeres = deletions_t2t_coordinates_telomeres.as_df()
deletions_t2t_coordinates_telomeres = deletions_t2t_coordinates_telomeres[deletions_t2t_coordinates_telomeres['Distance'] <1 ]
deletions_t2t_coordinates_telomeres
# Mean CN 
deletions_t2t_mean_individual_telomeres= deletions_t2t_coordinates_telomeres['LP6005441-DNA_G07'].mean()



#### SDs 
deletions_t2t_coordinates_sds = t2t_deletions_gb.nearest(t2t_sds)
deletions_t2t_coordinates_sds = deletions_t2t_coordinates_sds.as_df()
deletions_t2t_coordinates_sds = deletions_t2t_coordinates_sds[deletions_t2t_coordinates_sds['Distance'] <1 ]
# Mean CN 
deletions_t2t_mean_individual_sds= deletions_t2t_coordinates_sds['LP6005441-DNA_G07'].mean()


In [132]:
#Duplications

In [133]:
#### Genes 
duplications_t2t_coordinates_genes = t2t_duplications_gb.nearest(t2t_genes)
duplications_t2t_coordinates_genes = duplications_t2t_coordinates_genes.as_df()
duplications_t2t_coordinates_genes = duplications_t2t_coordinates_genes[duplications_t2t_coordinates_genes['Distance'] <1 ]
# Mean CN 
duplications_t2t_mean_individual_genes= duplications_t2t_coordinates_genes['LP6005441-DNA_G07'].mean()



#### Telomeres
duplications_t2t_coordinates_telomeres = t2t_duplications_gb.nearest(t2t_telomeres)
duplications_t2t_coordinates_telomeres = duplications_t2t_coordinates_telomeres.as_df()
duplications_t2t_coordinates_telomeres = duplications_t2t_coordinates_telomeres[duplications_t2t_coordinates_telomeres['Distance'] <1 ]
duplications_t2t_coordinates_telomeres
# Mean CN 
duplications_t2t_mean_individual_telomeres= duplications_t2t_coordinates_telomeres['LP6005441-DNA_G07'].mean()



#### SDs 
duplications_t2t_coordinates_sds = t2t_duplications_gb.nearest(t2t_sds)
duplications_t2t_coordinates_sds = duplications_t2t_coordinates_sds.as_df()
duplications_t2t_coordinates_sds = duplications_t2t_coordinates_sds[duplications_t2t_coordinates_sds['Distance'] <1 ]
# Mean CN 
duplications_t2t_mean_individual_sds= duplications_t2t_coordinates_sds['LP6005441-DNA_G07'].mean()


<h2> T2T CNVnator

In [134]:
header_list = ['Type', 'Position', 'Lenght', 'RD', 'T-TEST-EVAL', 'GAUSSIAN-EVAL', 'T-TEST-EVAL(MIDDLE)','GAUSSIAN-EVAL(MIDDLE)', 'FRAQ-READ-QUALITY', 't', 't2']

t2t_data = pd.read_csv('/Users/luciabazan/t2t_LP6005441-DNA_G07_cnvs.tsv', sep='\t' , names=header_list)
t2t_data


t2t_data = t2t_data[t2t_data['T-TEST-EVAL'] <= 0.05]
t2t_data

cnvnator_data = t2t_data

#SEPARATE COORDINATES
coordinates = cnvnator_data['Position'].str.split(':', expand=True)
CHROMOSOME = coordinates[0]

split_position = coordinates[1].str.split('-', expand=True)
START = split_position[0]
STOP = split_position[1]

cnvnator_style= pd.DataFrame()
cnvnator_style['Chromosome'] = CHROMOSOME
cnvnator_style['Start'] = START
cnvnator_style['End'] = STOP
cnvnator_style['CNV_value'] = cnvnator_data['RD']


cnvnator_style['Start'] = cnvnator_style['Start'].astype(int)
cnvnator_style['End'] = cnvnator_style['End'].astype(int)
cnvnator_style['CNV_value'] = cnvnator_style['CNV_value'].astype(float)

#cnvnator_style['Type'] = cnvnator_data['Type']

cnvnator_style['CNV_value'] = cnvnator_style['CNV_value'] * 2

cnvnator_style['Chromosome'] = cnvnator_style['Chromosome'].str.replace('chr','')

t2t_cnvnator = cnvnator_style
t2t_cnvnator['Length_cnvr'] = t2t_cnvnator['End'] - t2t_cnvnator['Start']
t2t_cnvnator = t2t_cnvnator[t2t_cnvnator['Chromosome'] != 'M']
t2t_cnvnator = t2t_cnvnator[t2t_cnvnator['Chromosome'] != 'X']
t2t_cnvnator = t2t_cnvnator[t2t_cnvnator['Chromosome'] != 'Y']
t2t_cnvnator

,Chromosome,Start,End,CNV_value,Length_cnvr
0,1,25001,45000,3.3208,19999
1,1,48001,78000,3.2608,29999
2,1,88001,96000,0.5038,7999
6,1,2133001,2162000,7.1520,28999
8,1,2193001,2223000,5.9894,29999
...,...,...,...,...,...
3408,22,24379001,24409000,3.0316,29999
3409,22,24409001,24427000,1.0380,17999
3410,22,24430001,24463000,1.1298,32999
3412,22,39109001,39114000,0.0000,4999


In [135]:
t2t_deletions_cnvnator = t2t_cnvnator[t2t_cnvnator['CNV_value'] < 2]
t2t_deletions_cnvnator_mean = t2t_deletions_cnvnator['CNV_value'].mean()
t2t_duplications_cnvnator = t2t_cnvnator[t2t_cnvnator['CNV_value'] > 2]
t2t_duplications_cnvnator_mean = t2t_duplications_cnvnator['CNV_value'].mean()
t2t_duplications_cnvnator = PyRanges(t2t_duplications_cnvnator)
t2t_deletions_cnvnator = PyRanges(t2t_deletions_cnvnator)

t2t_cnvnator= PyRanges(t2t_cnvnator)
t2t_cnvnator

/opt/anaconda3/lib/python3.11/site-packages/pyranges/methods/init.py:45: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  return {k: v for k, v in df.groupby(grpby_key)}
/opt/anaconda3/lib/python3.11/site-packages/pyranges/methods/init.py:45: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  return {k: v for k, v in df.groupby(grpby_key)}
/opt/anaconda3/lib/python3.11/site-packages/pyranges/methods/init.py:45: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default

,Chromosome,Start,End,CNV_value,Length_cnvr
0,1,25001,45000,3.3208,19999
1,1,48001,78000,3.2608,29999
2,1,88001,96000,0.5038,7999
3,1,2133001,2162000,7.1520,28999
4,1,2193001,2223000,5.9894,29999
...,...,...,...,...,...
1850,22,24379001,24409000,3.0316,29999
1851,22,24409001,24427000,1.0380,17999
1852,22,24430001,24463000,1.1298,32999
1853,22,39109001,39114000,0.0000,4999


In [136]:
#### t2t genes 
t2t_cnvnator_coordinates_genes = t2t_cnvnator.nearest(t2t_genes)
t2t_cnvnator_coordinates_genes = t2t_cnvnator_coordinates_genes.as_df()
t2t_cnvnator_coordinates_genes = t2t_cnvnator_coordinates_genes[t2t_cnvnator_coordinates_genes['Distance'] <1 ]

## T2T telomeres
t2t_cnvnator_coordinates_telomeres = t2t_cnvnator.nearest(t2t_telomeres)
t2t_cnvnator_coordinates_telomeres = t2t_cnvnator_coordinates_telomeres.as_df()
t2t_cnvnator_coordinates_telomeres = t2t_cnvnator_coordinates_telomeres[t2t_cnvnator_coordinates_telomeres['Distance'] <1 ]

#SDs 
t2t_cnvnator_coordinates_sds = t2t_cnvnator.nearest(t2t_sds)
t2t_cnvnator_coordinates_sds = t2t_cnvnator_coordinates_sds.as_df()
t2t_cnvnator_coordinates_sds = t2t_cnvnator_coordinates_sds[t2t_cnvnator_coordinates_sds['Distance'] <1 ]



In [137]:
#Deletions
#### Genes 
deletions_t2t_cnvnator_coordinates_genes = t2t_deletions_cnvnator.nearest(t2t_genes)
deletions_t2t_cnvnator_coordinates_genes = deletions_t2t_cnvnator_coordinates_genes.as_df()
deletions_t2t_cnvnator_coordinates_genes = deletions_t2t_cnvnator_coordinates_genes[deletions_t2t_cnvnator_coordinates_genes['Distance'] <1 ]
# Mean CN 
deletions_t2t_cnvnator_coordinates_genes.replace(2, np.nan, inplace=True)
deletions_t2t_cnvnator_mean_individual_genes= deletions_t2t_cnvnator_coordinates_genes['CNV_value'].mean()
deletions_t2t_cnvnator_coordinates_genes.replace(np.nan,2, inplace=True)


#### Telomeres
deletions_t2t_cnvnator_coordinates_telomeres = t2t_deletions_cnvnator.nearest(t2t_telomeres)
deletions_t2t_cnvnator_coordinates_telomeres = deletions_t2t_cnvnator_coordinates_telomeres.as_df()
deletions_t2t_cnvnator_coordinates_telomeres = deletions_t2t_cnvnator_coordinates_telomeres[deletions_t2t_cnvnator_coordinates_telomeres['Distance'] <1 ]
deletions_t2t_cnvnator_coordinates_telomeres
# Mean CN 
deletions_t2t_cnvnator_coordinates_telomeres.replace(2, np.nan, inplace=True)
deletions_t2t_cnvnator_mean_individual_telomeres= deletions_t2t_cnvnator_coordinates_telomeres['CNV_value'].mean()
deletions_t2t_cnvnator_coordinates_telomeres.replace(np.nan,2, inplace=True)


#### SDs 
deletions_t2t_cnvnator_coordinates_sds = t2t_deletions_cnvnator.nearest(t2t_sds)
deletions_t2t_cnvnator_coordinates_sds = deletions_t2t_cnvnator_coordinates_sds.as_df()
deletions_t2t_cnvnator_coordinates_sds = deletions_t2t_cnvnator_coordinates_sds[deletions_t2t_cnvnator_coordinates_sds['Distance'] <1 ]
# Mean CN 
deletions_t2t_cnvnator_coordinates_sds.replace(2, np.nan, inplace=True)
deletions_t2t_cnvnator_mean_individual_sds= deletions_t2t_cnvnator_coordinates_sds['CNV_value'].mean()
deletions_t2t_cnvnator_coordinates_sds.replace(np.nan,2, inplace=True)


In [138]:
deletions_t2t_cnvnator_mean_individual_sds

0.981063976945245

In [139]:
#Duplications
#### Genes 
duplications_t2t_cnvnator_coordinates_genes = t2t_duplications_cnvnator.nearest(t2t_genes)
duplications_t2t_cnvnator_coordinates_genes = duplications_t2t_cnvnator_coordinates_genes.as_df()
duplications_t2t_cnvnator_coordinates_genes = duplications_t2t_cnvnator_coordinates_genes[duplications_t2t_cnvnator_coordinates_genes['Distance'] <1 ]
# Mean CN 
duplications_t2t_cnvnator_coordinates_genes.replace(2, np.nan, inplace=True)
duplications_t2t_cnvnator_mean_individual_genes= duplications_t2t_cnvnator_coordinates_genes['CNV_value'].mean()
duplications_t2t_cnvnator_coordinates_genes.replace(np.nan,2, inplace=True)


#### Telomeres
duplications_t2t_cnvnator_coordinates_telomeres = t2t_duplications_cnvnator.nearest(t2t_telomeres)
duplications_t2t_cnvnator_coordinates_telomeres = duplications_t2t_cnvnator_coordinates_telomeres.as_df()
duplications_t2t_cnvnator_coordinates_telomeres = duplications_t2t_cnvnator_coordinates_telomeres[duplications_t2t_cnvnator_coordinates_telomeres['Distance'] <1 ]
duplications_t2t_cnvnator_coordinates_telomeres
# Mean CN 
duplications_t2t_cnvnator_coordinates_telomeres.replace(2, np.nan, inplace=True)
duplications_t2t_cnvnator_mean_individual_telomeres= duplications_t2t_cnvnator_coordinates_telomeres['CNV_value'].mean()
duplications_t2t_cnvnator_coordinates_telomeres.replace(np.nan,2, inplace=True)




#### SDs 
duplications_t2t_cnvnator_coordinates_sds = t2t_duplications_cnvnator.nearest(t2t_sds)
duplications_t2t_cnvnator_coordinates_sds = duplications_t2t_cnvnator_coordinates_sds.as_df()
duplications_t2t_cnvnator_coordinates_sds = duplications_t2t_cnvnator_coordinates_sds[duplications_t2t_cnvnator_coordinates_sds['Distance'] <1 ]
# Mean CN 

duplications_t2t_cnvnator_coordinates_sds.replace(2, np.nan, inplace=True)
duplications_t2t_cnvnator_mean_individual_sds= duplications_t2t_cnvnator_coordinates_sds['CNV_value'].mean()
duplications_t2t_cnvnator_coordinates_sds.replace(np.nan,2, inplace=True)
duplications_t2t_cnvnator_mean_individual_sds

3.5933225225225227

<h1> GRCh37

In [140]:
## Anotations
grch37_genes = pd.read_csv('../annotation_files/grch37-genes.csv')
grch37_telomeres = pd.read_csv('../annotation_files/grch37-telomers-centromers.csv')
grch37_sds = pd.read_csv('../annotation_files/grch37-sds.csv')
#grch37_sds = grch37_sds[['#chr1', 'start1', 'end1']]


## Read and merge files 
anotation = pd.read_csv('../data/SGDP_anotation.csv', sep=',', encoding='latin-1')

ids_hg19 = pd.DataFrame()
ids_hg19['Sample'] = anotation['3-Illumina_ID']
ids_hg19['Region'] = anotation['10-Region']


#t2t_sds.columns =['Chromosome','Start','End']
#t2t_sds
grch37_genes.columns = ['#"bin"', 'name', 'Chromosome', 'strand', 'Start', 'End', 'cdsStart',
       'cdsEnd', 'exonCount', 'exonStarts', 'exonEnds', 'score', 'name2',
       'cdsStartStat', 'cdsEndStat', 'exonFrames'] 
grch37_telomeres = grch37_telomeres[grch37_telomeres['type'] == 'telomere']
grch37_sds.columns = ['#"bin"', 'Chromosome', 'Start', 'End', 'name', 'score', 'strand',
       'otherChrom', 'otherStart', 'otherEnd', 'otherSize', 'uid',
       'posBasesHit', 'testResult', 'verdict', 'chits', 'ccov', 'alignfile',
       'alignL', 'indelN', 'indelS', 'alignB', 'matchB', 'mismatchB',
       'transitionsB', 'transversionsB', 'fracMatch', 'fracMatchIndel', 'jcK',
       'k2K']

In [141]:
data = pd.read_csv("../call_cnvr_cnvnator/individual_cnv_cnvr.txt", sep='\t')
data = pd.read_csv("../data/cnvnator_cnvs.csv")
data.columns = ['Sample_ID','Chromosome', 'Start', 'End','CNV_Value','Type','RD','Lenght', 'T-TEST-EVAL','GAUSSIAN-EVAL',]
data = data.dropna()#.describe()
data['Length_cnvr'] = data['End'] - data['Start'] 
sample_old = data[data['Sample_ID'] == 'LP6005441-DNA_G07']
#sample_old = old_assembly[old_assembly['Sample_ID'] == 'LP6005441-DNA_B10']
#sample_old = sample_old[['Chromosome', 'Start', 'End', 'RD']]
sample_old['RD'] = sample_old['RD'] * 2
#sample_old['Chromosome'] = str('chr')+sample_old['Chromosome']
    #sample_old.to_csv('LP6005441-DNA_B10_grch37.bed', index=False, header=False, sep='\t')
    #list(sample_old.columns)

#sample_old

sample_old = sample_old[~sample_old['RD'].isin([2])]

sample_old = sample_old[sample_old['Chromosome'] != 'M']
sample_old = sample_old[sample_old['Chromosome'] != 'X']
sample_old = sample_old[sample_old['Chromosome'] != 'Y']

sample_old 

/var/folders/05/4pfxtbrs5459ytz3j1jhcgt00000gn/T/ipykernel_91372/4141503609.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample_old['RD'] = sample_old['RD'] * 2


,Sample_ID,Chromosome,Start,End,CNV_Value,Type,RD,Lenght,T-TEST-EVAL,GAUSSIAN-EVAL,Length_cnvr
51663,LP6005441-DNA_G07,1,234001,238300,3.0,duplication,4.170320,4300.0,2.965070e-10,1.407390e+02,4299
51664,LP6005441-DNA_G07,1,267101,317800,1.0,deletion,0.010921,50700.0,3.143440e-12,0.000000e+00,50699
51665,LP6005441-DNA_G07,1,471301,521400,1.0,deletion,0.000109,50100.0,3.181090e-12,0.000000e+00,50099
51666,LP6005441-DNA_G07,1,1010801,1014700,1.0,deletion,0.444038,3900.0,4.086470e-11,2.141070e-31,3899
51667,LP6005441-DNA_G07,1,4124401,4127800,1.0,deletion,1.013284,3400.0,4.687430e-11,1.314760e-38,3399
...,...,...,...,...,...,...,...,...,...,...,...
52386,LP6005441-DNA_G07,9,138194201,138197800,1.0,deletion,1.323550,3600.0,1.411230e-02,2.138410e+09,3599
52387,LP6005441-DNA_G07,9,138214101,138217600,1.0,deletion,1.019674,3500.0,4.553500e-11,1.185690e-11,3499
52388,LP6005441-DNA_G07,9,138479101,138480200,1.0,deletion,0.000000,1100.0,1.448840e-10,3.214710e-77,1099
52389,LP6005441-DNA_G07,9,139663601,139665300,1.0,deletion,0.604050,1700.0,1.392830e-03,2.771230e-02,1699


In [142]:
###### Keeping only samples that are on old cnvs 
sample_old = sample_old.merge(ids_hg19, right_on=['Sample'], left_on=['Sample_ID'])

#REMOVE SAMPLES WITH ABERRATIONS
#sample_old.columns = ['Chromosome', 'Start', 'End', 'CNVR_ID', 'Sample_iD', 'CNV_start', 'CNV_end', 'CNV_Value','Length', 'Length_cnvr','Sample','Region']
sample_old 

,Sample_ID,Chromosome,Start,End,CNV_Value,Type,RD,Lenght,T-TEST-EVAL,GAUSSIAN-EVAL,Length_cnvr,Sample,Region
0,LP6005441-DNA_G07,1,234001,238300,3.0,duplication,4.170320,4300.0,2.965070e-10,1.407390e+02,4299,LP6005441-DNA_G07,America
1,LP6005441-DNA_G07,1,267101,317800,1.0,deletion,0.010921,50700.0,3.143440e-12,0.000000e+00,50699,LP6005441-DNA_G07,America
2,LP6005441-DNA_G07,1,471301,521400,1.0,deletion,0.000109,50100.0,3.181090e-12,0.000000e+00,50099,LP6005441-DNA_G07,America
3,LP6005441-DNA_G07,1,1010801,1014700,1.0,deletion,0.444038,3900.0,4.086470e-11,2.141070e-31,3899,LP6005441-DNA_G07,America
4,LP6005441-DNA_G07,1,4124401,4127800,1.0,deletion,1.013284,3400.0,4.687430e-11,1.314760e-38,3399,LP6005441-DNA_G07,America
...,...,...,...,...,...,...,...,...,...,...,...,...,...
723,LP6005441-DNA_G07,9,138194201,138197800,1.0,deletion,1.323550,3600.0,1.411230e-02,2.138410e+09,3599,LP6005441-DNA_G07,America
724,LP6005441-DNA_G07,9,138214101,138217600,1.0,deletion,1.019674,3500.0,4.553500e-11,1.185690e-11,3499,LP6005441-DNA_G07,America
725,LP6005441-DNA_G07,9,138479101,138480200,1.0,deletion,0.000000,1100.0,1.448840e-10,3.214710e-77,1099,LP6005441-DNA_G07,America
726,LP6005441-DNA_G07,9,139663601,139665300,1.0,deletion,0.604050,1700.0,1.392830e-03,2.771230e-02,1699,LP6005441-DNA_G07,America


In [143]:
#change to df when using telomeres and centromeres filtration
final_cnv_grch37 = sample_old.pivot_table(index=["Chromosome", "Start", "End", 'Length_cnvr'], 
                    columns='Sample', 
                    values='RD',
                        fill_value=2).reset_index()


final_cnv_grch37

Sample,Chromosome,Start,End,Length_cnvr,LP6005441-DNA_G07
0,1,234001,238300,4299,4.170320
1,1,267101,317800,50699,0.010921
2,1,471301,521400,50099,0.000109
3,1,1010801,1014700,3899,0.444038
4,1,4124401,4127800,3399,1.013284
...,...,...,...,...,...
723,9,138194201,138197800,3599,1.323550
724,9,138214101,138217600,3499,1.019674
725,9,138479101,138480200,1099,0.000000
726,9,139663601,139665300,1699,0.604050


In [144]:
grch37_deletions_cnvnator = final_cnv_grch37[final_cnv_grch37['LP6005441-DNA_G07'] < 2]
grch37_deletions_cnvnator_mean = grch37_deletions_cnvnator['LP6005441-DNA_G07'].mean()

grch37_duplications_cnvnator = final_cnv_grch37[final_cnv_grch37['LP6005441-DNA_G07'] > 2]
grch37_duplications_cnvnator_mean = grch37_duplications_cnvnator['LP6005441-DNA_G07'].mean()

grch37_duplications_cnvnator = PyRanges(grch37_duplications_cnvnator)
grch37_deletions_cnvnator = PyRanges(grch37_deletions_cnvnator)


final_cnv_ranges_grch37 = PyRanges(final_cnv_grch37)
grch37_deletions_cnvnator

/opt/anaconda3/lib/python3.11/site-packages/pyranges/methods/init.py:45: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  return {k: v for k, v in df.groupby(grpby_key)}
/opt/anaconda3/lib/python3.11/site-packages/pyranges/methods/init.py:45: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  return {k: v for k, v in df.groupby(grpby_key)}
/opt/anaconda3/lib/python3.11/site-packages/pyranges/methods/init.py:45: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default

Sample,Chromosome,Start,End,Length_cnvr,LP6005441-DNA_G07
0,1,267101,317800,50699,0.010921
1,1,471301,521400,50099,0.000109
2,1,1010801,1014700,3899,0.444038
3,1,4124401,4127800,3399,1.013284
4,1,8360201,8364000,3799,0.017621
...,...,...,...,...,...
503,22,21880701,21887200,6499,0.925102
504,22,21899601,21905500,5899,1.209440
505,22,23478501,23479700,1199,0.186410
506,22,37143101,37147900,4799,0.057464


<h2> Genes

In [145]:
grch37_genes['Chromosome'] = grch37_genes['Chromosome'].str.replace('chr', '')
grch37_genes_coordinates = PyRanges(grch37_genes)

grch37_telomeres['Chromosome'] = grch37_telomeres['Chromosome'].str.replace('chr', '')
grch37_telomeres = PyRanges(grch37_telomeres)

grch37_sds['Chromosome'] = grch37_sds['Chromosome'].str.replace('chr', '')
grch37_sds = PyRanges(grch37_sds)

/opt/anaconda3/lib/python3.11/site-packages/pyranges/methods/init.py:45: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  return {k: v for k, v in df.groupby(grpby_key)}
/opt/anaconda3/lib/python3.11/site-packages/pyranges/methods/init.py:45: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  return {k: v for k, v in df.groupby(grpby_key)}
/opt/anaconda3/lib/python3.11/site-packages/pyranges/methods/init.py:45: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default

In [146]:
#Genes
grch37_coordinates_genes = final_cnv_ranges_grch37.nearest(grch37_genes_coordinates)
grch37_coordinates_genes = grch37_coordinates_genes.as_df()
grch37_coordinates_genes = grch37_coordinates_genes[grch37_coordinates_genes['Distance'] <1 ]
#grch37_coordinates_genes

#Telomeres
grch37_coordinates_telomeres = final_cnv_ranges_grch37.nearest(grch37_telomeres)
grch37_coordinates_telomeres = grch37_coordinates_telomeres.as_df()
grch37_coordinates_telomeres = grch37_coordinates_telomeres[grch37_coordinates_telomeres['Distance'] <1 ]

#SDs
grch37_coordinates_sds = final_cnv_ranges_grch37.nearest(grch37_sds)
grch37_coordinates_sds = grch37_coordinates_sds.as_df()
grch37_coordinates_sds = grch37_coordinates_sds[grch37_coordinates_sds['Distance'] <1 ]

In [147]:
#grch37_genes = len(grch37_coordinates_genes[['Chromosome','Start', 'End']].drop_duplicates())
#grch37_genes

In [148]:
## Deletions
#### Genes 
deletions_grch37_cnvnator_coordinates_genes = grch37_deletions_cnvnator.nearest(grch37_genes_coordinates)
deletions_grch37_cnvnator_coordinates_genes = deletions_grch37_cnvnator_coordinates_genes.as_df()
deletions_grch37_cnvnator_coordinates_genes = deletions_grch37_cnvnator_coordinates_genes[deletions_grch37_cnvnator_coordinates_genes['Distance'] <1 ]
# Mean CN 
deletions_grch37_cnvnator_coordinates_genes.replace(2, np.nan, inplace=True)
deletions_grch37_cnvnator_mean_individual_genes= deletions_grch37_cnvnator_coordinates_genes['LP6005441-DNA_G07'].mean()
deletions_grch37_cnvnator_coordinates_genes.replace(np.nan,2, inplace=True)


#### Telomeres
deletions_grch37_cnvnator_coordinates_telomeres = grch37_deletions_cnvnator.nearest(grch37_telomeres)
deletions_grch37_cnvnator_coordinates_telomeres = deletions_grch37_cnvnator_coordinates_telomeres.as_df()
deletions_grch37_cnvnator_coordinates_telomeres = deletions_grch37_cnvnator_coordinates_telomeres[deletions_grch37_cnvnator_coordinates_telomeres['Distance'] <1 ]
deletions_grch37_cnvnator_coordinates_telomeres
# Mean CN 
deletions_grch37_cnvnator_coordinates_telomeres.replace(2, np.nan, inplace=True)
deletions_grch37_cnvnator_mean_individual_telomeres= deletions_grch37_cnvnator_coordinates_telomeres['LP6005441-DNA_G07'].mean()
deletions_grch37_cnvnator_coordinates_telomeres.replace(np.nan,2, inplace=True)


#### SDs 
deletions_grch37_cnvnator_coordinates_sds = grch37_deletions_cnvnator.nearest(grch37_sds)
deletions_grch37_cnvnator_coordinates_sds = deletions_grch37_cnvnator_coordinates_sds.as_df()
deletions_grch37_cnvnator_coordinates_sds = deletions_grch37_cnvnator_coordinates_sds[deletions_grch37_cnvnator_coordinates_sds['Distance'] <1 ]
# Mean CN 
deletions_grch37_cnvnator_coordinates_sds.replace(2, np.nan, inplace=True)
deletions_grch37_cnvnator_mean_individual_sds= deletions_grch37_cnvnator_coordinates_sds['LP6005441-DNA_G07'].mean()
deletions_grch37_cnvnator_coordinates_sds.replace(np.nan,2, inplace=True)


In [149]:
## Duplications
#### Genes 
duplications_grch37_cnvnator_coordinates_genes = grch37_duplications_cnvnator.nearest(grch37_genes_coordinates)
duplications_grch37_cnvnator_coordinates_genes = duplications_grch37_cnvnator_coordinates_genes.as_df()
duplications_grch37_cnvnator_coordinates_genes = duplications_grch37_cnvnator_coordinates_genes[duplications_grch37_cnvnator_coordinates_genes['Distance'] <1 ]
# Mean CN 
duplications_grch37_cnvnator_coordinates_genes.replace(2, np.nan, inplace=True)
duplications_grch37_cnvnator_mean_individual_genes= duplications_grch37_cnvnator_coordinates_genes['LP6005441-DNA_G07'].mean()
duplications_grch37_cnvnator_coordinates_genes.replace(np.nan,2, inplace=True)


#### Telomeres
duplications_grch37_cnvnator_coordinates_telomeres = grch37_duplications_cnvnator.nearest(grch37_telomeres)
duplications_grch37_cnvnator_coordinates_telomeres = duplications_grch37_cnvnator_coordinates_telomeres.as_df()
duplications_grch37_cnvnator_coordinates_telomeres = duplications_grch37_cnvnator_coordinates_telomeres[duplications_grch37_cnvnator_coordinates_telomeres['Distance'] <1 ]
duplications_grch37_cnvnator_coordinates_telomeres
# Mean CN 
duplications_grch37_cnvnator_coordinates_telomeres.replace(2, np.nan, inplace=True)
duplications_grch37_cnvnator_mean_individual_telomeres= duplications_grch37_cnvnator_coordinates_telomeres['LP6005441-DNA_G07'].mean()
duplications_grch37_cnvnator_coordinates_telomeres.replace(np.nan,2, inplace=True)


#### SDs 
duplications_grch37_cnvnator_coordinates_sds = grch37_duplications_cnvnator.nearest(grch37_sds)
duplications_grch37_cnvnator_coordinates_sds = duplications_grch37_cnvnator_coordinates_sds.as_df()
duplications_grch37_cnvnator_coordinates_sds = duplications_grch37_cnvnator_coordinates_sds[duplications_grch37_cnvnator_coordinates_sds['Distance'] <1 ]
# Mean CN 
duplications_grch37_cnvnator_coordinates_sds.replace(2, np.nan, inplace=True)
duplications_grch37_cnvnator_mean_individual_sds= duplications_grch37_cnvnator_coordinates_sds['LP6005441-DNA_G07'].mean()
duplications_grch37_cnvnator_coordinates_sds.replace(np.nan,2, inplace=True)

<h2> Peter Sudmant data 

In [150]:
sudmant = pd.read_csv('/Users/luciabazan/Downloads/sudmant_data.csv')
sudmant = sudmant[['contig','start','end','AMR_Maya_HGDP00855_F']]
sudmant.columns =['Chromosome', 'Start', 'End', 'CNV_value']


#sudmant['Chromosome'] = sudmant['Chromosome'].str.replace('chr','hs')
n_uncalled = np.sum(sudmant==-1,1)
sudmant = sudmant[n_uncalled==0]

sudmant['Length_cnvr'] = sudmant['End'] - sudmant['Start'] 
sudmant = sudmant[~sudmant['CNV_value'].isin([2])]


sudmant['Chromosome'] = sudmant['Chromosome'].str.replace('chr', '')
sudmant = sudmant[sudmant['Chromosome'] != 'M']
sudmant = sudmant[sudmant['Chromosome'] != 'X']
sudmant = sudmant[sudmant['Chromosome'] != 'Y']


grch37_deletions_sudmant = sudmant[sudmant['CNV_value'] < 3]
grch37_deletions_sudmant_mean = grch37_deletions_sudmant['CNV_value'].mean()

grch37_duplications_sudmant = sudmant[sudmant['CNV_value'] > 1]
grch37_duplications_sudmant_mean = grch37_duplications_sudmant['CNV_value'].mean()

grch37_duplications_sudmant = PyRanges(grch37_duplications_sudmant)
grch37_deletions_sudmant = PyRanges(grch37_deletions_sudmant)


sudmant = PyRanges(sudmant)
sudmant

/opt/anaconda3/lib/python3.11/site-packages/pyranges/methods/init.py:45: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  return {k: v for k, v in df.groupby(grpby_key)}
/opt/anaconda3/lib/python3.11/site-packages/pyranges/methods/init.py:45: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  return {k: v for k, v in df.groupby(grpby_key)}
/opt/anaconda3/lib/python3.11/site-packages/pyranges/methods/init.py:45: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default

,Chromosome,Start,End,CNV_value,Length_cnvr
0,1,12048,177033,22,164985
1,1,12048,89237,10,77189
2,1,228416,267160,24,38744
3,1,471226,471326,10,100
4,1,318047,471226,21,153179
...,...,...,...,...,...
4311,22,42973857,42980083,4,6226
4312,22,42973707,42973857,4,150
4313,22,49013936,49016261,5,2325
4314,22,51223959,51224959,4,1000


In [151]:
#Genes
peter_coordinates_genes = sudmant.nearest(grch37_genes_coordinates)
peter_coordinates_genes = peter_coordinates_genes.as_df()
peter_coordinates_genes = peter_coordinates_genes[peter_coordinates_genes['Distance'] <1 ]
#grch37_coordinates_genes

#Telomeres
peter_coordinates_telomeres = sudmant.nearest(grch37_telomeres)
peter_coordinates_telomeres = peter_coordinates_telomeres.as_df()
peter_coordinates_telomeres = peter_coordinates_telomeres[peter_coordinates_telomeres['Distance'] <1 ]

#SDs
peter_coordinates_sds = sudmant.nearest(grch37_sds)
peter_coordinates_sds = peter_coordinates_sds.as_df()
peter_coordinates_sds = peter_coordinates_sds[peter_coordinates_sds['Distance'] <1 ]

In [152]:
peter_coordinates_sds#[['Chromosome','Start', 'End']].drop_duplicates()

,Chromosome,Start,End,CNV_value,Length_cnvr,"#""bin""",Start_b,End_b,name,score,...,alignB,matchB,mismatchB,transitionsB,transversionsB,fracMatch,fracMatchIndel,jcK,k2K,Distance
0,1,12048,177033,22,164985,585,10000,87112,chr15:102446355,0,...,74269,73742,527,332,195,0.992904,0.991956,0.007130,0.007133,0
1,1,12048,89237,10,77189,585,10000,87112,chr15:102446355,0,...,74269,73742,527,332,195,0.992904,0.991956,0.007130,0.007133,0
2,1,228416,267160,24,38744,586,227417,238719,chr17:81176011,0,...,11249,11065,184,123,61,0.983643,0.981984,0.016538,0.016561,0
3,1,471226,471326,10,100,73,317719,471368,chr5:180744334,0,...,153555,153444,111,75,36,0.999277,0.999121,0.000723,0.000723,0
4,1,318047,471226,21,153179,73,317719,471368,chr5:180744334,0,...,153555,153444,111,75,36,0.999277,0.999121,0.000723,0.000723,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4290,22,42973857,42980083,4,6226,912,42962426,42978456,chr22:42902112,0,...,13422,13031,391,245,146,0.970869,0.967122,0.029712,0.029771,0
4291,22,42973707,42973857,4,150,912,42962426,42978456,chr22:42902112,0,...,13422,13031,391,245,146,0.970869,0.967122,0.029712,0.029771,0
4292,22,49013936,49016261,5,2325,958,49014584,49015948,chr15:22514585,0,...,1331,1270,61,44,17,0.954170,0.949888,0.047290,0.047556,0
4293,22,51223959,51224959,4,1000,975,51198214,51236901,chr2:114370005,0,...,38593,38225,368,242,126,0.990465,0.989260,0.009597,0.009604,0


In [153]:
## Deletions
#### Genes 
deletions_grch37_sudmant_coordinates_genes = grch37_deletions_sudmant.nearest(grch37_genes_coordinates)
deletions_grch37_sudmant_coordinates_genes = deletions_grch37_sudmant_coordinates_genes.as_df()
deletions_grch37_sudmant_coordinates_genes = deletions_grch37_sudmant_coordinates_genes[deletions_grch37_sudmant_coordinates_genes['Distance'] <1 ]
# Mean CN 
deletions_grch37_sudmant_coordinates_genes.replace(2, np.nan, inplace=True)
deletions_grch37_sudmant_mean_individual_genes= deletions_grch37_sudmant_coordinates_genes['CNV_value'].mean()
deletions_grch37_sudmant_coordinates_genes.replace(np.nan,2, inplace=True)


#### Telomeres
deletions_grch37_sudmant_coordinates_telomeres = grch37_deletions_sudmant.nearest(grch37_telomeres)
deletions_grch37_sudmant_coordinates_telomeres = deletions_grch37_sudmant_coordinates_telomeres.as_df()
deletions_grch37_sudmant_coordinates_telomeres = deletions_grch37_sudmant_coordinates_telomeres[deletions_grch37_sudmant_coordinates_telomeres['Distance'] <1 ]
deletions_grch37_sudmant_coordinates_telomeres
# Mean CN 
deletions_grch37_sudmant_coordinates_telomeres.replace(2, np.nan, inplace=True)
deletions_grch37_sudmant_mean_individual_telomeres= deletions_grch37_sudmant_coordinates_telomeres['CNV_value'].mean()
deletions_grch37_sudmant_coordinates_telomeres.replace(np.nan,2, inplace=True)


#### SDs 
deletions_grch37_sudmant_coordinates_sds = grch37_deletions_sudmant.nearest(grch37_sds)
deletions_grch37_sudmant_coordinates_sds = deletions_grch37_sudmant_coordinates_sds.as_df()
deletions_grch37_sudmant_coordinates_sds = deletions_grch37_sudmant_coordinates_sds[deletions_grch37_sudmant_coordinates_sds['Distance'] <1 ]
# Mean CN 
deletions_grch37_sudmant_coordinates_sds.replace(2, np.nan, inplace=True)
deletions_grch37_sudmant_mean_individual_sds= deletions_grch37_sudmant_coordinates_sds['CNV_value'].mean()
deletions_grch37_sudmant_coordinates_sds.replace(np.nan,2, inplace=True)


In [154]:
## Duplications
#### Genes 
duplications_grch37_sudmant_coordinates_genes = grch37_duplications_sudmant.nearest(grch37_genes_coordinates)
duplications_grch37_sudmant_coordinates_genes = duplications_grch37_sudmant_coordinates_genes.as_df()
duplications_grch37_sudmant_coordinates_genes = duplications_grch37_sudmant_coordinates_genes[duplications_grch37_sudmant_coordinates_genes['Distance'] <1 ]
# Mean CN 
duplications_grch37_sudmant_coordinates_genes.replace(2, np.nan, inplace=True)
duplications_grch37_sudmant_mean_individual_genes= duplications_grch37_sudmant_coordinates_genes['CNV_value'].mean()
duplications_grch37_sudmant_coordinates_genes.replace(np.nan,2, inplace=True)


#### Telomeres
duplications_grch37_sudmant_coordinates_telomeres = grch37_duplications_sudmant.nearest(grch37_telomeres)
duplications_grch37_sudmant_coordinates_telomeres = duplications_grch37_sudmant_coordinates_telomeres.as_df()
duplications_grch37_sudmant_coordinates_telomeres = duplications_grch37_sudmant_coordinates_telomeres[duplications_grch37_sudmant_coordinates_telomeres['Distance'] <1 ]
duplications_grch37_sudmant_coordinates_telomeres
# Mean CN 
duplications_grch37_sudmant_coordinates_telomeres.replace(2, np.nan, inplace=True)
duplications_grch37_sudmant_mean_individual_telomeres= duplications_grch37_sudmant_coordinates_telomeres['CNV_value'].mean()
duplications_grch37_sudmant_coordinates_telomeres.replace(np.nan,2, inplace=True)


#### SDs 
duplications_grch37_sudmant_coordinates_sds = grch37_duplications_sudmant.nearest(grch37_sds)
duplications_grch37_sudmant_coordinates_sds = duplications_grch37_sudmant_coordinates_sds.as_df()
duplications_grch37_sudmant_coordinates_sds = duplications_grch37_sudmant_coordinates_sds[duplications_grch37_sudmant_coordinates_sds['Distance'] <1 ]
# Mean CN 
duplications_grch37_sudmant_coordinates_sds.replace(2, np.nan, inplace=True)
duplications_grch37_sudmant_mean_individual_sds= duplications_grch37_sudmant_coordinates_sds['CNV_value'].mean()
duplications_grch37_sudmant_coordinates_sds.replace(np.nan,2, inplace=True)


GRCh37 Duplications

In [155]:
literature = {'Data': ['GRCh37', '  Genes','  Telomeres','  SDs','T2T-CHM13', '  Genes','  Telomeres','  SDs'
             ], 
     'Literature':[len(sudmant), 
                  len(peter_coordinates_genes[['Chromosome','Start', 'End']].drop_duplicates()),
                  len(peter_coordinates_telomeres[['Chromosome','Start', 'End']].drop_duplicates()), 
                  len(peter_coordinates_sds[['Chromosome','Start', 'End']].drop_duplicates()), 
                  len(final_cnv_ranges), 
                  len(coordinates_genes[['Chromosome','Start', 'End']].drop_duplicates()),
                  len(coordinates_telomeres[['Chromosome','Start', 'End']].drop_duplicates()), 
                  len(coordinates_sds[['Chromosome','Start', 'End']].drop_duplicates()), ], 
     'Mean size(Bp) Literature':[round(sudmant.as_df()['Length_cnvr'].mean()), 
                      round(peter_coordinates_genes['Length_cnvr'].mean()), 
                      peter_coordinates_telomeres['Length_cnvr'].mean(), 
                      round(peter_coordinates_sds['Length_cnvr'].mean()), 
                      round(final_cnv['Length_cnvr'].mean()),
                      round(coordinates_genes['Length_cnvr'].mean()),
                      round(coordinates_telomeres['Length_cnvr'].mean()), 
                      round(coordinates_sds['Length_cnvr'].mean())],
    'Literature  Deletions':[len(grch37_deletions_sudmant), 
                              len(deletions_grch37_sudmant_coordinates_genes[['Chromosome','Start', 'End']].drop_duplicates()), 
                                     len(deletions_grch37_sudmant_coordinates_telomeres[['Chromosome','Start', 'End']].drop_duplicates()),
                                     len(deletions_grch37_sudmant_coordinates_sds[['Chromosome','Start', 'End']].drop_duplicates()),
                                     len(t2t_deletions_gb),
                                     len(deletions_t2t_coordinates_genes[['Chromosome','Start', 'End']].drop_duplicates()),
                                     len(deletions_t2t_coordinates_telomeres[['Chromosome','Start', 'End']].drop_duplicates()),
                                     len(deletions_t2t_coordinates_sds[['Chromosome','Start', 'End']].drop_duplicates())],
     
     'Literature Mean CN Deletions':[grch37_deletions_sudmant_mean, 
                                     deletions_grch37_sudmant_mean_individual_genes, 
                                     deletions_grch37_sudmant_mean_individual_telomeres,
                                     deletions_grch37_sudmant_mean_individual_sds,
                                     t2t_deletions_gb_mean,
                                     deletions_t2t_mean_individual_genes,
                                     deletions_t2t_mean_individual_telomeres,
                                     deletions_t2t_mean_individual_sds
                                    ],

     
     'Literature  Duplications':[len(grch37_duplications_sudmant), 
                              len(duplications_grch37_sudmant_coordinates_genes[['Chromosome','Start', 'End']].drop_duplicates()), 
                                     len(duplications_grch37_sudmant_coordinates_telomeres[['Chromosome','Start', 'End']].drop_duplicates()),
                                     len(duplications_grch37_sudmant_coordinates_sds[['Chromosome','Start', 'End']].drop_duplicates()),
                                     len(t2t_duplications_gb),
                                     len(duplications_t2t_coordinates_genes[['Chromosome','Start', 'End']].drop_duplicates()),
                                     len(duplications_t2t_coordinates_telomeres[['Chromosome','Start', 'End']].drop_duplicates()),
                                     len(duplications_t2t_coordinates_sds[['Chromosome','Start', 'End']].drop_duplicates())],
     'Literature Mean CN Duplications':[grch37_duplications_sudmant_mean,
                                       duplications_grch37_sudmant_mean_individual_genes,
                                       duplications_grch37_sudmant_mean_individual_telomeres,
                                       duplications_grch37_sudmant_mean_individual_sds,
                                       t2t_duplications_gb_mean,
                                        duplications_t2t_mean_individual_genes,
                                        duplications_t2t_mean_individual_telomeres,
                                        duplications_t2t_mean_individual_sds],
    
    }
df = pd.DataFrame(data=literature)
df

,Data,Literature,Mean size(Bp) Literature,Literature Deletions,Literature Mean CN Deletions,Literature Duplications,Literature Mean CN Duplications
0,GRCh37,4316,32098.0,333,0.726727,3983,10.283204
1,Genes,3087,39603.0,180,0.738889,2907,10.553492
2,Telomeres,0,NaN,0,NaN,0,NaN
3,SDs,3704,36046.0,20,0.850000,3684,10.779045
4,T2T-CHM13,92834,4636.0,7174,0.980485,85660,31.152393
5,Genes,63950,3086.0,4179,0.986360,59771,20.399742
6,Telomeres,43,126802.0,0,NaN,43,37.186047
7,SDs,66176,5227.0,51,1.000000,66125,38.810541


In [156]:
d = {'Data': ['GRCh37', '  Genes','  Telomeres','  SDs','T2T-CHM13', '  Genes','  Telomeres','  SDs'
             ], 
     'CNVNator': [len(final_cnv_grch37), 
               len(grch37_coordinates_genes[['Chromosome','Start', 'End']].drop_duplicates()), 
               len(grch37_coordinates_telomeres[['Chromosome','Start', 'End']].drop_duplicates()), 
               len(grch37_coordinates_sds[['Chromosome','Start', 'End']].drop_duplicates()),  
               len(t2t_cnvnator), 
               len(t2t_cnvnator_coordinates_genes[['Chromosome','Start', 'End']].drop_duplicates()), 
               len(t2t_cnvnator_coordinates_telomeres[['Chromosome','Start', 'End']].drop_duplicates()), 
               len(t2t_cnvnator_coordinates_sds[['Chromosome','Start', 'End']].drop_duplicates())
              ], 
     'Mean size(Bp)':[final_cnv_grch37['Length_cnvr'].mean().round(), 
                      grch37_coordinates_genes['Length_cnvr'].mean().round(), 
                      grch37_coordinates_telomeres['Length_cnvr'].mean().round(), 
                      grch37_coordinates_sds['Length_cnvr'].mean().round(), 
                      t2t_cnvnator.as_df()['Length_cnvr'].mean().round(),
                      t2t_cnvnator_coordinates_genes['Length_cnvr'].mean().round(),
                      t2t_cnvnator_coordinates_telomeres['Length_cnvr'].mean().round(), 
                      t2t_cnvnator_coordinates_sds['Length_cnvr'].mean().round()],
     
     'Deletions':[len(grch37_deletions_cnvnator), 
                              len(deletions_grch37_cnvnator_coordinates_genes[['Chromosome','Start', 'End']].drop_duplicates()), 
                                     len(deletions_grch37_cnvnator_coordinates_telomeres[['Chromosome','Start', 'End']].drop_duplicates()),
                                     len(deletions_grch37_cnvnator_coordinates_sds[['Chromosome','Start', 'End']].drop_duplicates()),
                                     len(t2t_deletions_cnvnator),
                                     len(deletions_t2t_cnvnator_coordinates_genes[['Chromosome','Start', 'End']].drop_duplicates()),
                                     len(deletions_t2t_cnvnator_coordinates_telomeres[['Chromosome','Start', 'End']].drop_duplicates()),
                                     len(deletions_t2t_cnvnator_coordinates_sds[['Chromosome','Start', 'End']].drop_duplicates())],

     'CNVnator Mean CN Deletions':[grch37_deletions_cnvnator_mean,
                                   deletions_grch37_cnvnator_mean_individual_genes,
                                   deletions_grch37_cnvnator_mean_individual_telomeres,
                                   deletions_grch37_cnvnator_mean_individual_sds,
                                   t2t_deletions_cnvnator_mean, 
                                   deletions_t2t_cnvnator_mean_individual_genes,
                                   deletions_t2t_cnvnator_mean_individual_telomeres,
                                   deletions_t2t_cnvnator_mean_individual_sds],

    'Duplications':[len(grch37_duplications_cnvnator), 
                              len(duplications_grch37_cnvnator_coordinates_genes[['Chromosome','Start', 'End']].drop_duplicates()), 
                                     len(duplications_grch37_cnvnator_coordinates_telomeres[['Chromosome','Start', 'End']].drop_duplicates()),
                                     len(duplications_grch37_cnvnator_coordinates_sds[['Chromosome','Start', 'End']].drop_duplicates()),
                                     len(t2t_duplications_cnvnator),
                                     len(duplications_t2t_cnvnator_coordinates_genes[['Chromosome','Start', 'End']].drop_duplicates()),
                                     len(duplications_t2t_cnvnator_coordinates_telomeres[['Chromosome','Start', 'End']].drop_duplicates()),
                                     len(duplications_t2t_cnvnator_coordinates_sds[['Chromosome','Start', 'End']].drop_duplicates())],

     'CNVnator Mean CN Duplications':[grch37_duplications_cnvnator_mean,
                                      duplications_grch37_cnvnator_mean_individual_genes,
                                      duplications_grch37_cnvnator_mean_individual_telomeres,
                                      duplications_grch37_cnvnator_mean_individual_sds,
                                      t2t_duplications_cnvnator_mean,
                                      duplications_t2t_cnvnator_mean_individual_genes,
                                      duplications_t2t_cnvnator_mean_individual_telomeres,
                                      duplications_t2t_cnvnator_mean_individual_sds
                                      
                                     ],


    
    }
df = pd.DataFrame(data=d)
df

,Data,CNVNator,Mean size(Bp),Deletions,CNVnator Mean CN Deletions,Duplications,CNVnator Mean CN Duplications
0,GRCh37,728,18512.0,508,0.773704,220,4.375335
1,Genes,383,23790.0,272,0.786549,111,4.271579
2,Telomeres,3,47899.0,3,0.099335,0,NaN
3,SDs,430,27020.0,234,0.852147,196,4.463832
4,T2T-CHM13,1855,66703.0,1420,0.894372,435,3.969565
5,Genes,386,64483.0,224,0.997807,162,3.344081
6,Telomeres,5,101599.0,3,1.049467,2,4.295300
7,SDs,569,59707.0,347,0.981064,222,3.593323


In [157]:
len(peter_coordinates_sds[['Chromosome','Start', 'End']].drop_duplicates())

3704